In [1]:
# Install required packages
!pip install torch torch-geometric scikit-learn rdkit pandas -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.2/36.2 MB 64.5 MB/s eta 0:00:00


In [2]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Imports
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv   
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from tqdm import tqdm
import os
import time

# RDKit for SMILES fingerprints
from rdkit import Chem
from rdkit.Chem import AllChem, MACCSkeys
from rdkit import DataStructs

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


#### Learning_rate = 0.005          *before we use 0.001 and 0.005 but there was no improvement until we used 0.01*


In [34]:
Fingerprint_type = 'morgan'
N_bit = 512
Hidden_dimensions = 128
Included_dimensions = 128
Dropout = 0.5
Batch_size = 128
Learning_rate = 0.005        # before we use 0.001 and 0.005 but there was no improvement until we used 0.01
Decay_weight = 0.04
Number_Eproch = 500
Seeds = 42
Aggregator = 'mean'
torch.manual_seed(Seeds)
np.random.seed(Seeds)
if torch.cuda.is_available():
    torch.cuda.manual_seed(Seeds)

print("\n" + "="*80)
print("CONFIGURATION")
print("="*80)
print(f"Model: GCN + MLP")
print(f"Aggregator: {Aggregator.upper()}")
print(f"Fingerprint: {Fingerprint_type.upper()} ({N_bit}-bit)")
print(f"Hidden Dim: {Hidden_dimensions}")
print(f"Dropout: {Dropout}")
print(f"Learning Rate: {Learning_rate}")
print(f"Weight Decay: {Decay_weight}")
print(f"Epochs: {Number_Eproch}")
print(f"Batch Size: {Batch_size}")
print("="*80)


CONFIGURATION
Model: GCN + MLP
Aggregator: MEAN
Fingerprint: MORGAN (512-bit)
Hidden Dim: 128
Dropout: 0.5
Learning Rate: 0.005
Weight Decay: 0.04
Epochs: 500
Batch Size: 128


In [ ]:
DATA_PATH = '/content/drive/MyDrive/MLHygnn/BaseLine/GCN/data/'
DATA_PATH_SMILES = '/content/drive/MyDrive/MLHygnn/BaseLine/'

print("Loading data from Google Drive...")

# Load CSV files
train_df = pd.read_csv(DATA_PATH + 'train_positive.csv')
val_df = pd.read_csv(DATA_PATH + 'val_positive.csv')
test_df = pd.read_csv(DATA_PATH + 'test_positive.csv')

print(f"\nTrain data preview:")
print(train_df.head())
print(f"\nDataset sizes:")
print(f"  Training: {len(train_df):,} pairs")
print(f"  Validation: {len(val_df):,} pairs")
print(f"  Test: {len(test_df):,} pairs")

Loading data from Google Drive...

Train data preview:
  Drug1_ID Drug2_ID  Label
0  DB01097  DB05219     47
1  DB00547  DB00784     49
2  DB00623  DB01365     61
3  DB00328  DB09027     73
4  DB00742  DB00955     57

Dataset sizes:
  Training: 153,489 pairs
  Validation: 19,188 pairs
  Test: 19,200 pairs


## 6. Load Drug SMILES

In [6]:
# Load drug SMILES
drug_smiles_df = pd.read_csv(DATA_PATH_SMILES + 'Drugs_with_Smiles.csv')

print(f"\nDrug SMILES loaded: {len(drug_smiles_df)} drugs")
print(drug_smiles_df.head())


Drug SMILES loaded: 1709 drugs
  DrugBank_ID                                             SMILES
0     DB00006  CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@...
1     DB00014  CC(C)C[C@H](NC(=O)[C@@H](COC(C)(C)C)NC(=O)[C@H...
2     DB00027  CC(C)C[C@@H](NC(=O)CNC(=O)[C@@H](NC=O)C(C)C)C(...
3     DB00035  NC(=O)CC[C@@H]1NC(=O)[C@H](CC2=CC=CC=C2)NC(=O)...
4     DB00080  CCCCCCCCCC(=O)N[C@@H](CC1=CNC2=C1C=CC=C2)C(=O)...


## 7. Extract Molecular Fingerprints

### We use Morgan Fingerprint (2,512)

In [ ]:
# Get unique drugs
all_drugs = pd.concat([
    train_df['Drug1_ID'], train_df['Drug2_ID'],
    val_df['Drug1_ID'], val_df['Drug2_ID'],
    test_df['Drug1_ID'], test_df['Drug2_ID']
]).unique()

print(f"\n🔬 Extracting {Fingerprint_type} fingerprints...")
print(f"Total unique drugs: {len(all_drugs)}")

def smiles_to_fingerprint(smiles, fp_type='morgan', n_bits=512):
    """Convert SMILES to molecular fingerprint"""
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            return None

        if fp_type == 'morgan':
            fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=n_bits)
        elif fp_type == 'maccs':
            fp = MACCSkeys.GenMACCSKeys(mol)
        else:
            raise ValueError(f"Unknown fingerprint type: {fp_type}")

        arr = np.zeros((0,), dtype=np.int8)
        DataStructs.ConvertToNumpyArray(fp, arr)
        return arr
    except:
        return None

# Extract fingerprints
drug_to_fp = {}
drug_to_idx = {drug: idx for idx, drug in enumerate(all_drugs)}

for drug_id in tqdm(all_drugs, desc="Extracting fingerprints"):
    smiles_row = drug_smiles_df[drug_smiles_df['DrugBank_ID'] == drug_id]
    if len(smiles_row) > 0:
        smiles = smiles_row.iloc[0]['SMILES']
        fp = smiles_to_fingerprint(smiles, fp_type=Fingerprint_type, n_bits=N_bit)
        if fp is not None:
            drug_to_fp[drug_id] = fp
        else:
            # Default fingerprint for failed cases
            drug_to_fp[drug_id] = np.zeros(N_bit, dtype=np.int8)
    else:
        drug_to_fp[drug_id] = np.zeros(N_bit, dtype=np.int8)

# Create feature matrix
num_drugs = len(all_drugs)
drug_features = np.zeros((num_drugs, N_bit), dtype=np.float32)

for drug_id, idx in drug_to_idx.items():
    if drug_id in drug_to_fp:
        drug_features[idx] = drug_to_fp[drug_id]

drug_features = torch.FloatTensor(drug_features).to(device)

print(f"\nFingerprints extracted!")
print(f"Feature matrix shape: {drug_features.shape}")
print(f"Device: {drug_features.device}")

## 8. Prepare Graph Data

In [35]:
def prepare_pairs(df, drug_to_idx):
    """Convert dataframe to pair indices and labels"""
    pairs = []
    labels = []

    for _, row in df.iterrows():
        drug1 = row['Drug1_ID']
        drug2 = row['Drug2_ID']
        label = row['Label']

        if drug1 in drug_to_idx and drug2 in drug_to_idx:
            idx1 = drug_to_idx[drug1]
            idx2 = drug_to_idx[drug2]
            pairs.append([idx1, idx2])
            labels.append(label)

    return torch.LongTensor(pairs), torch.LongTensor(labels)

# Prepare training, validation, and test data
train_pairs, train_types = prepare_pairs(train_df, drug_to_idx)
val_pairs, val_types = prepare_pairs(val_df, drug_to_idx)
test_pairs, test_types = prepare_pairs(test_df, drug_to_idx)
# If labels are 1-86, convert to 0-85
if train_types.min() == 1:
    train_types = train_types - 1
    val_types = val_types - 1
    test_types = test_types - 1
    print(f"Converted labels from 1-86 to 0-85")
else:
    print("Labels already 0-based")
# Move to device
train_pairs = train_pairs.to(device)
train_types = train_types.to(device)
val_pairs = val_pairs.to(device)
val_types = val_types.to(device)
test_pairs = test_pairs.to(device)
test_types = test_types.to(device)

# Build edge index from training data (for GCN graph structure)
edge_index = torch.cat([train_pairs.T, train_pairs.T[[1, 0]]], dim=1)  # Bidirectional edges

print(f"\nData prepared!")
print(f"Training pairs: {len(train_pairs):,}")
print(f"Validation pairs: {len(val_pairs):,}")
print(f"Test pairs: {len(test_pairs):,}")
print(f"Edge index shape: {edge_index.shape}")
print(f"Number of classes: {train_types.max().item() + 1}")

Converted labels from 1-86 to 0-85

Data prepared!
Training pairs: 153,489
Validation pairs: 19,188
Test pairs: 19,200
Edge index shape: torch.Size([2, 306978])
Number of classes: 86


## 9. Define GCN + MLP Model

In [ ]:
class GCN_MLP(nn.Module):

    def __init__(self, input_dim, hidden_dim, num_classes, dropout=0.5):
        super(GCN_MLP, self).__init__()

        # Single GCN layer (matching 1-layer HGNN)
        self.gcn1 = GCNConv(input_dim, hidden_dim)

        # MLP for pair classification
        self.mlp = nn.Sequential(
            nn.Linear(hidden_dim * 4, hidden_dim * 2),  # 512 → 256
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim * 2, hidden_dim),       # 256 → 128
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, num_classes)           # 128 → 86
        )

        self.dropout = dropout

    def forward(self, x, edge_index, drug_pairs):

        # GCN Single layer
        x = self.gcn1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout, training=self.training)

        # Get embeddings for drug pairs
        h_i = x[drug_pairs[:, 0]]  # First drug in pair
        h_j = x[drug_pairs[:, 1]]  # Second drug in pair

        h_pair = torch.cat([
            h_i,                    # Drug 1 embedding
            h_j,                    # Drug 2 embedding
            h_i * h_j,              # Element-wise product
            torch.abs(h_i - h_j)    # Absolute difference
        ], dim=-1)

        # MLP classifier
        logits = self.mlp(h_pair)

        return logits

# Initialize model
model = GCN_MLP(
    input_dim=N_bit,
    hidden_dim=Hidden_dimensions,
    num_classes=86,
    dropout=Dropout
).to(device)

print("\n" + "="*80)
print("MODEL ARCHITECTURE")
print("="*80)
print(model)
print("="*80)

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"\nTotal parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")
print("="*80)


MODEL ARCHITECTURE
GCN_MLP(
  (gcn1): GCNConv(512, 128)
  (mlp): Sequential(
    (0): Linear(in_features=512, out_features=256, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=256, out_features=128, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.5, inplace=False)
    (6): Linear(in_features=128, out_features=86, bias=True)
  )
)

Total parameters: 240,982
Trainable parameters: 240,982


## 10. Training Setup

### We used class weight in both the training and verification phases because the model couldn't train when we only applied class weight during the verification phase. Therefore, we had to apply it in both phases to enable it to learn and train. This contrasts with our model, which was able to train without class weight 💪🥇.

In [ ]:
# Calculate class weights (EXACTLY like our HGNN)
type_counts = torch.bincount(train_types, minlength=86).float()
alpha = 0.5
class_weights = 1.0 / torch.pow(type_counts.clamp(min=1.0), alpha)
class_weights = class_weights / class_weights.mean()
class_weights = class_weights.to(device)

print("\n" + "="*80)
print("CLASS WEIGHTS")
print("="*80)
print(f"Class weights statistics (alpha={alpha}):")
print(f"  Min weight: {class_weights.min().item():.4f}")
print(f"  Max weight: {class_weights.max().item():.4f}")
print(f"  Mean weight: {class_weights.mean().item():.4f}")
print(f"  Sample counts - Min: {type_counts.min().int().item()}, Max: {type_counts.max().int().item()}")
print("="*80)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = torch.optim.Adam(model.parameters(), lr=Learning_rate, weight_decay=Decay_weight)

print(f"\nTraining setup complete!")
print(f"Optimizer: Adam (lr={Learning_rate}, weight_decay={Decay_weight})")
print(f"Loss: CrossEntropyLoss with class weights")


CLASS WEIGHTS
Class weights statistics (alpha=0.5):
  Min weight: 0.0387
  Max weight: 4.2745
  Mean weight: 1.0000
  Sample counts - Min: 4, Max: 48746

Training setup complete!
Optimizer: Adam (lr=0.005, weight_decay=0.04)
Loss: CrossEntropyLoss with class weights


## 11. Training and Evaluation Functions

In [38]:
def train_epoch(model, optimizer, criterion, pairs, labels, edge_index, x, batch_size=128):
    """Train for one epoch"""
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    # Shuffle training data
    perm = torch.randperm(len(pairs))

    for i in range(0, len(pairs), batch_size):
        batch_idx = perm[i:i+batch_size]
        batch_pairs = pairs[batch_idx]
        batch_labels = labels[batch_idx]

        optimizer.zero_grad()

        # Forward pass
        logits = model(x, edge_index, batch_pairs)
        loss = criterion(logits, batch_labels)

        # Backward pass
        loss.backward()
        optimizer.step()

        # Statistics
        total_loss += loss.item() * len(batch_pairs)
        _, predicted = torch.max(logits, 1)
        correct += (predicted == batch_labels).sum().item()
        total += len(batch_pairs)

    avg_loss = total_loss / total
    accuracy = correct / total

    return avg_loss, accuracy

@torch.no_grad()
def evaluate(model, pairs, labels, edge_index, x, criterion, class_weights, batch_size=128):
    """Evaluate model"""
    model.eval()
    total_loss = 0
    all_preds = []
    all_labels = []

    for i in range(0, len(pairs), batch_size):
        batch_pairs = pairs[i:i+batch_size]
        batch_labels = labels[i:i+batch_size]

        # Forward pass
        logits = model(x, edge_index, batch_pairs)
        loss = criterion(logits, batch_labels)

        total_loss += loss.item() * len(batch_pairs)

        _, predicted = torch.max(logits, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(batch_labels.cpu().numpy())

    avg_loss = total_loss / len(pairs)

    # Calculate metrics
    metrics = {
        'accuracy': accuracy_score(all_labels, all_preds),
        'f1_micro': f1_score(all_labels, all_preds, average='micro', zero_division=0),
        'f1_macro': f1_score(all_labels, all_preds, average='macro', zero_division=0),
        'precision': precision_score(all_labels, all_preds, average='micro', zero_division=0),
        'recall': recall_score(all_labels, all_preds, average='micro', zero_division=0)
    }

    return metrics, avg_loss

# RAM usage calculator
def calculate_ram_usage():
    """Calculate RAM usage in GB"""
    import psutil
    process = psutil.Process()
    return process.memory_info().rss / 1024 / 1024 / 1024



## 12. Train Model experment-0
N_bit = 512,
Hidden_dimensions = 128,
Included_dimensions = 128,
Dropout = 0.5,
Batch_size = 128,
Learning_rate = 0.005,     
Decay_weight = 0.04,
Number_Eproch = 500,
Seeds = 42,
alph=0.5,


## belowe the train and output

In [39]:

print("\n" + "="*80)
print(f"FULL TRAINING GCN+MLP ({Number_Eproch} EPOCHS) - GPU ACCELERATED")
print("="*80)

# Get RAM usage before training
ram_before = calculate_ram_usage()
print(f"RAM usage before training: {ram_before:.2f} GB")
print(f"Device: {device}")
print(f"Training data: {len(train_pairs):,} pairs\n")

best_val_loss = float('inf')
best_epoch = 0
patience =40
patience_counter = 0
start_time = time.time()

for epoch in range(Number_Eproch):
    epoch_start_time = time.time()

    # Training phase
    train_loss, train_acc = train_epoch(
        model, optimizer, criterion,
        train_pairs, train_types,
        edge_index, drug_features,
        batch_size=Batch_size
    )

    # Validation phase
    val_metrics, val_loss = evaluate(
        model, val_pairs, val_types,
        edge_index, drug_features,
        criterion, class_weights,
        batch_size=Batch_size
    )

    epoch_time = time.time() - epoch_start_time

    # Check for improvement
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_epoch = epoch
        patience_counter = 0
        # Save best model
        torch.save(model.state_dict(), '/content/best_gcn_mlp_model.pth')
        print(f"New best: Epoch {epoch} - Val Loss: {val_loss:.4f} (Time: {epoch_time:.1f}s)")
    else:
        patience_counter += 1

    # Print every 10 epochs
    if epoch % 10 == 0:
        print(f"Epoch {epoch}: loss: {train_loss:.4f}, val_loss: {val_loss:.4f} (best: {best_val_loss:.4f}, patience: {patience_counter})")

    # Early stopping
    if patience_counter >= patience:
        print(f"\nEarly stopping at epoch {epoch}")
        break

total_time = time.time() - start_time

print("\n" + "="*80)
print("TRAINING COMPLETED")
print("="*80)
print(f"Best epoch: {best_epoch}")
print(f"Best validation loss: {best_val_loss:.4f}")
print(f"Total time: {total_time/60:.2f} minutes")
print(f"Average time per epoch: {total_time/max(epoch+1, 1):.1f} seconds")
print("="*80)


FULL TRAINING GCN+MLP (500 EPOCHS) - GPU ACCELERATED
RAM usage before training: 1.66 GB
Device: cuda
Training data: 153,489 pairs

New best: Epoch 0 - Val Loss: 3.8220 (Time: 9.9s)
Epoch 0: loss: 3.9123, val_loss: 3.8220 (best: 3.8220, patience: 0)
New best: Epoch 3 - Val Loss: 3.8095 (Time: 10.0s)
New best: Epoch 8 - Val Loss: 3.8042 (Time: 9.9s)
Epoch 10: loss: 3.9060, val_loss: 3.9008 (best: 3.8042, patience: 2)
New best: Epoch 13 - Val Loss: 3.7735 (Time: 10.0s)
Epoch 20: loss: 3.9150, val_loss: 3.9197 (best: 3.7735, patience: 7)
New best: Epoch 22 - Val Loss: 3.7620 (Time: 10.1s)
Epoch 30: loss: 3.9027, val_loss: 3.7902 (best: 3.7620, patience: 8)
Epoch 40: loss: 3.9123, val_loss: 3.8284 (best: 3.7620, patience: 18)
Epoch 50: loss: 3.9076, val_loss: 3.7993 (best: 3.7620, patience: 28)
Epoch 60: loss: 3.9070, val_loss: 3.7917 (best: 3.7620, patience: 38)

Early stopping at epoch 62

TRAINING COMPLETED
Best epoch: 22
Best validation loss: 3.7620
Total time: 10.49 minutes
Average ti

## 12. Train Model experment-1
N_bit = 512,
Hidden_dimensions = 128,
Included_dimensions = 128,
Dropout = 0.6,
Batch_size = 128,
Learning_rate = 0.05,     
Decay_weight = 0.01,
Number_Eproch = 500,
Seeds = 42,
alph=0.4,

In [18]:

print("\n" + "="*80)
print(f"FULL TRAINING GCN+MLP ({Number_Eproch} EPOCHS) - GPU ACCELERATED")
print("="*80)

# Get RAM usage before training
ram_before = calculate_ram_usage()
print(f"RAM usage before training: {ram_before:.2f} GB")
print(f"Device: {device}")
print(f"Training data: {len(train_pairs):,} pairs\n")

best_val_loss = float('inf')
best_epoch = 0
patience =40
patience_counter = 0
start_time = time.time()

for epoch in range(Number_Eproch):
    epoch_start_time = time.time()

    # Training phase
    train_loss, train_acc = train_epoch(
        model, optimizer, criterion,
        train_pairs, train_types,
        edge_index, drug_features,
        batch_size=Batch_size
    )

    # Validation phase
    val_metrics, val_loss = evaluate(
        model, val_pairs, val_types,
        edge_index, drug_features,
        criterion, class_weights,
        batch_size=Batch_size
    )

    epoch_time = time.time() - epoch_start_time

    # Check for improvement
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_epoch = epoch
        patience_counter = 0
        # Save best model
        torch.save(model.state_dict(), '/content/best_gcn_mlp_model.pth')
        print(f"New best: Epoch {epoch} - Val Loss: {val_loss:.4f} (Time: {epoch_time:.1f}s)")
    else:
        patience_counter += 1

    # Print every 10 epochs
    if epoch % 10 == 0:
        print(f"Epoch {epoch}: loss: {train_loss:.4f}, val_loss: {val_loss:.4f} (best: {best_val_loss:.4f}, patience: {patience_counter})")

    # Early stopping
    if patience_counter >= patience:
        print(f"\nEarly stopping at epoch {epoch}")
        break

total_time = time.time() - start_time

print("\n" + "="*80)
print("TRAINING COMPLETED")
print("="*80)
print(f"Best epoch: {best_epoch}")
print(f"Best validation loss: {best_val_loss:.4f}")
print(f"Total time: {total_time/60:.2f} minutes")
print(f"Average time per epoch: {total_time/max(epoch+1, 1):.1f} seconds")
print("="*80)


FULL TRAINING GCN+MLP (500 EPOCHS) - GPU ACCELERATED
RAM usage before training: 1.65 GB
Device: cuda
Training data: 153,489 pairs

New best: Epoch 0 - Val Loss: 3.6385 (Time: 12.1s)
Epoch 0: loss: 3.7330, val_loss: 3.6385 (best: 3.6385, patience: 0)
New best: Epoch 3 - Val Loss: 3.6048 (Time: 10.0s)
Epoch 10: loss: 3.7646, val_loss: 3.6791 (best: 3.6048, patience: 7)
Epoch 20: loss: 3.7380, val_loss: 3.6163 (best: 3.6048, patience: 17)
Epoch 30: loss: 3.7869, val_loss: 3.6572 (best: 3.6048, patience: 27)
Epoch 40: loss: 3.8045, val_loss: 3.6455 (best: 3.6048, patience: 37)

Early stopping at epoch 43

TRAINING COMPLETED
Best epoch: 3
Best validation loss: 3.6048
Total time: 7.38 minutes
Average time per epoch: 10.1 seconds


## 12. Train Model experment-2 

N_bit = 512,
Hidden_dimensions = 128,
Included_dimensions = 128,
Dropout = 0.5,
Batch_size = 128,
Learning_rate = 0.01 ,      
Decay_weight = 0.001,
Number_Eproch = 500,
Seeds = 42,
alph=0.3

In [12]:
print("\n" + "="*80)
print(f"FULL TRAINING GCN+MLP ({Number_Eproch} EPOCHS) - GPU ACCELERATED")
print("="*80)

# Get RAM usage before training
ram_before = calculate_ram_usage()
print(f"RAM usage before training: {ram_before:.2f} GB")
print(f"Device: {device}")
print(f"Training data: {len(train_pairs):,} pairs\n")

best_val_loss = float('inf')
best_epoch = 0
patience = 100
patience_counter = 0
start_time = time.time()

for epoch in range(Number_Eproch):
    epoch_start_time = time.time()

    # Training phase
    train_loss, train_acc = train_epoch(
        model, optimizer, criterion,
        train_pairs, train_types,
        edge_index, drug_features,
        batch_size=Batch_size
    )

    # Validation phase
    val_metrics, val_loss = evaluate(
        model, val_pairs, val_types,
        edge_index, drug_features,
        criterion, class_weights,
        batch_size=Batch_size
    )

    epoch_time = time.time() - epoch_start_time

    # Check for improvement
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_epoch = epoch
        patience_counter = 0
        # Save best model
        torch.save(model.state_dict(), '/content/best_gcn_mlp_model.pth')
        print(f"New best: Epoch {epoch} - Val Loss: {val_loss:.4f} (Time: {epoch_time:.1f}s)")
    else:
        patience_counter += 1

    # Print every 10 epochs
    if epoch % 10 == 0:
        print(f"Epoch {epoch}: loss: {train_loss:.4f}, val_loss: {val_loss:.4f} (best: {best_val_loss:.4f}, patience: {patience_counter})")

    # Early stopping
    if patience_counter >= patience:
        print(f"\nEarly stopping at epoch {epoch}")
        break

total_time = time.time() - start_time

print("\n" + "="*80)
print("TRAINING COMPLETED")
print("="*80)
print(f"Best epoch: {best_epoch}")
print(f"Best validation loss: {best_val_loss:.4f}")
print(f"Total time: {total_time/60:.2f} minutes")
print(f"Average time per epoch: {total_time/max(epoch+1, 1):.1f} seconds")
print("="*80)


FULL TRAINING GCN+MLP (500 EPOCHS) - GPU ACCELERATED
RAM usage before training: 1.21 GB
Device: cuda
Training data: 153,489 pairs

New best: Epoch 0 - Val Loss: 3.0629 (Time: 10.7s)
Epoch 0: loss: 3.2326, val_loss: 3.0629 (best: 3.0629, patience: 0)
New best: Epoch 6 - Val Loss: 3.0594 (Time: 9.9s)
New best: Epoch 7 - Val Loss: 3.0308 (Time: 10.0s)
New best: Epoch 9 - Val Loss: 3.0290 (Time: 10.0s)
Epoch 10: loss: 3.1736, val_loss: 3.0915 (best: 3.0290, patience: 1)
Epoch 20: loss: 3.3268, val_loss: 3.2967 (best: 3.0290, patience: 11)
Epoch 30: loss: 3.3244, val_loss: 3.2929 (best: 3.0290, patience: 21)
Epoch 40: loss: 3.3256, val_loss: 3.2925 (best: 3.0290, patience: 31)
Epoch 50: loss: 3.3258, val_loss: 3.2932 (best: 3.0290, patience: 41)
Epoch 60: loss: 3.3255, val_loss: 3.2984 (best: 3.0290, patience: 51)
Epoch 70: loss: 3.3248, val_loss: 3.2909 (best: 3.0290, patience: 61)
Epoch 80: loss: 3.3262, val_loss: 3.2899 (best: 3.0290, patience: 71)
Epoch 90: loss: 3.3262, val_loss: 3.29

In [ ]:
torch.save(model.state_dict(), DATA_PATH + 'best_gcn_mlp_model.pth')

## 13. Evaluate on Test Set

In [ ]:
# Load best model
model.load_state_dict(torch.load('/content/best_gcn_mlp_model.pth'))

# Evaluate on test set
test_metrics, test_loss = evaluate(
    model, test_pairs, test_types,
    edge_index, drug_features,
    criterion, class_weights,
    batch_size=Batch_size
)

print("\n" + "="*80)
print("FINAL TEST SET RESULTS (GCN + MLP)")
print("="*80)
print(f"Test Loss:  {test_loss:.4f}")
print(f"Accuracy:   {test_metrics['accuracy']:.4f}")
print(f"F1 (Micro): {test_metrics['f1_micro']:.4f}")
print(f"F1 (Macro): {test_metrics['f1_macro']:.4f}")
print(f"Precision:  {test_metrics['precision']:.4f}")
print(f"Recall:     {test_metrics['recall']:.4f}")
print("="*80)

## 15. Save Results

In [ ]:
SAVE_PATH =  DATA_PATH+"result/"
os.makedirs(SAVE_PATH, exist_ok=True)

# Save model
torch.save(model.state_dict(), SAVE_PATH + 'gcn_mlp_morgan512_model.pth')

# Save results
import json
results = {
    'model': 'GCN+MLP',
    'features': 'Morgan 512',
    'aggregator': Aggregator,
    'best_epoch': best_epoch,
    'best_val_loss': best_val_loss,
    'test_metrics': test_metrics,
    'test_loss': test_loss,
    'training_time_minutes': total_time / 60,
    'hyperparameters': {
        'hidden_dim': Hidden_dimensions,
        'dropout': Dropout,
        'learning_rate': Learning_rate,
        'weight_decay': Decay_weight,
        'batch_size': Batch_size,
        'epochs': Number_Eproch
    }
}

with open(SAVE_PATH + 'gcn_mlp_results.json', 'w') as f:
    json.dump(results, f, indent=2)

print(f"\nResults saved to: {SAVE_PATH}")
print("   - gcn_mlp_morgan512_model.pth")
print("   - gcn_mlp_results.json")